In [1090]:
#liabraries 
import pandas as pd
import numpy as np
import ast 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.stem.porter import PorterStemmer  


In [1091]:
#read the csv file
songs = pd.read_csv('song.csv')
song = pd.DataFrame(songs)


In [1092]:
songs.shape

(1116, 16)

In [1093]:
song.shape

(1116, 16)

In [1094]:
song.drop_duplicates(subset='title', keep='first', inplace=True)
#update the index
song.reset_index(inplace=True)  



In [1095]:
song.shape

(1037, 17)

In [1096]:
song.head(-1)

,index,title,artist,top genre,year released,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,top year,artist type
0,0,STARSTRUKK (feat. Katy Perry),3OH!3,dance pop,2009,140,81,61,-6,23,23,203,0,6,70,2010,Duo
1,1,My First Kiss (feat. Ke$ha),3OH!3,dance pop,2010,138,89,68,-4,36,83,192,1,8,68,2010,Duo
2,2,I Need A Dollar,Aloe Blacc,pop soul,2010,95,48,84,-7,9,96,243,20,3,72,2010,Solo
3,3,Airplanes (feat. Hayley Williams of Paramore),B.o.B,atl hip hop,2010,93,87,66,-4,4,38,180,11,12,80,2010,Solo
4,4,Nothin' on You (feat. Bruno Mars),B.o.B,atl hip hop,2010,104,85,69,-6,9,74,268,39,5,79,2010,Solo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,1110,How You'll Be Remembered,Aron Wright,acoustic pop,2019,78,11,35,-15,10,12,259,94,3,22,2019,Solo
1032,1111,Drive-In,Adam Ragsdale,pop,2019,145,36,45,-11,11,25,220,77,4,15,2019,Solo
1033,1112,Only One,The Belle Game,vancouver indie,2018,86,74,45,-5,12,41,277,0,4,14,2018,Band/Group
1034,1113,Seasonal,Ailbhe Reddy,irish indie,2016,84,33,33,-9,11,21,209,65,3,14,2016,Solo


In [1097]:
#delete the first index coloumn
song.drop(['index'], axis=1, inplace=True)

In [1098]:
#view the title coloumm
song['title'].head()



0                    STARSTRUKK (feat. Katy Perry)
1                      My First Kiss (feat. Ke$ha)
2                                  I Need A Dollar
3    Airplanes (feat. Hayley Williams of Paramore)
4                Nothin' on You (feat. Bruno Mars)
Name: title, dtype: object

In [1099]:
#list of feat singers 
feat_singers = song['title'].apply(lambda x: x.split('(feat.')[-1].split(')')[0].strip() if '(feat.' in x else '')


In [1100]:
len(feat_singers)

1037

In [1101]:
feat_singers

0                        Katy Perry
1                             Ke$ha
2                                  
3       Hayley Williams of Paramore
4                        Bruno Mars
                   ...             
1032                               
1033                               
1034                               
1035                               
1036                               
Name: title, Length: 1037, dtype: object

In [1102]:
songs['title'].head()

0                    STARSTRUKK (feat. Katy Perry)
1                      My First Kiss (feat. Ke$ha)
2                                  I Need A Dollar
3    Airplanes (feat. Hayley Williams of Paramore)
4                Nothin' on You (feat. Bruno Mars)
Name: title, dtype: object

In [1103]:
#remove  feat singers from the title coloumn
song['title'] = song['title'].apply(lambda x: x.split('(feat.')[-2].split(')')[0].strip() if '(feat.' in x else x)


In [1104]:
song['title'].head()

0         STARSTRUKK
1      My First Kiss
2    I Need A Dollar
3          Airplanes
4     Nothin' on You
Name: title, dtype: object

In [1105]:
song.head()

,title,artist,top genre,year released,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop,top year,artist type
0,STARSTRUKK,3OH!3,dance pop,2009,140,81,61,-6,23,23,203,0,6,70,2010,Duo
1,My First Kiss,3OH!3,dance pop,2010,138,89,68,-4,36,83,192,1,8,68,2010,Duo
2,I Need A Dollar,Aloe Blacc,pop soul,2010,95,48,84,-7,9,96,243,20,3,72,2010,Solo
3,Airplanes,B.o.B,atl hip hop,2010,93,87,66,-4,4,38,180,11,12,80,2010,Solo
4,Nothin' on You,B.o.B,atl hip hop,2010,104,85,69,-6,9,74,268,39,5,79,2010,Solo


In [1106]:

singerz = song['artist'] + ', ' + feat_singers
singerz = singerz.str.rstrip(', ')

# Check if there are more than two singers
more_than_two = feat_singers.str.count(',') >= 2

singerz = singerz.where(feat_singers.notnull(), song['artist'])
singerz = singerz.where(~more_than_two, singerz + ' and')

singerz.head()



0                     3OH!3, Katy Perry
1                          3OH!3, Ke$ha
2                            Aloe Blacc
3    B.o.B, Hayley Williams of Paramore
4                     B.o.B, Bruno Mars
dtype: object

In [1107]:
song['artist'] = song['artist'].str.replace(' ', '')
feat_singers = [singer.replace(" ", "") for singer in feat_singers]



In [1108]:
len(feat_singers)

1037

In [1109]:
#add the feat singers list to the artist coloumn 
song['artist'] = song['artist'] + ' ' + feat_singers
singer = song['artist']


In [1110]:
#list of coloumn heading 
song.columns

Index(['title', 'artist', 'top genre', 'year released', 'bpm', 'nrgy', 'dnce',
       'dB', 'live', 'val', 'dur', 'acous', 'spch', 'pop', 'top year',
       'artist type'],
      dtype='object')

In [1111]:
#drop the unneccesary coloumns year release, added, top year
song.drop(['year released',  'top year', 'dur'], axis=1, inplace=True)



In [1112]:
song.head()

,title,artist,top genre,bpm,nrgy,dnce,dB,live,val,acous,spch,pop,artist type
0,STARSTRUKK,3OH!3 KatyPerry,dance pop,140,81,61,-6,23,23,0,6,70,Duo
1,My First Kiss,3OH!3 Ke$ha,dance pop,138,89,68,-4,36,83,1,8,68,Duo
2,I Need A Dollar,AloeBlacc,pop soul,95,48,84,-7,9,96,20,3,72,Solo
3,Airplanes,B.o.B HayleyWilliamsofParamore,atl hip hop,93,87,66,-4,4,38,11,12,80,Solo
4,Nothin' on You,B.o.B BrunoMars,atl hip hop,104,85,69,-6,9,74,39,5,79,Solo


In [1113]:
#turn the coloumns into smaller case 
song['artist'] = song['artist'].str.lower()
song['top genre'] = song['top genre'].str.lower()



In [1114]:
#check null values
song.isnull().sum()


title          0
artist         0
top genre      1
bpm            0
nrgy           0
dnce           0
dB             0
live           0
val            0
acous          0
spch           0
pop            0
artist type    0
dtype: int64

In [1115]:
#remove spacing 
song['top genre'] = song['top genre'].str.replace(' ', '')


In [1116]:
#check for duplicates titles
song['title'].duplicated().sum()
#return the duplicated titles
song[song['title'].duplicated(keep=False)].sort_values('title').head()
#remove the duplicates keepig the first one


,title,artist,top genre,bpm,nrgy,dnce,dB,live,val,acous,spch,pop,artist type
382,Bad,wale rihanna,gangsterrap,114,70,81,-7,11,45,31,7,67,Solo
409,Bad,davidguetta vassy,bigroom,128,97,61,-4,33,41,0,9,56,Solo
702,Location,khalid,pop,80,45,74,-11,9,33,33,43,83,Solo
875,Location,dave burnaboy,ukhiphop,110,50,81,-6,10,55,27,30,81,Solo
5,Magic,b.o.b riverscuomo,atlhiphop,82,93,55,-4,35,79,1,34,71,Solo


In [1117]:
song.shape

(1037, 13)

In [1118]:
song.shape

(1037, 13)

In [1119]:
#check for duplicates titles
song['title'].duplicated().sum()

6

In [1120]:
#join the following coloums 'top genre', 'val', 'nrgy', 'dnce','acous', 'spch', 'bpm', 'dur', 'dB', 'live', 'artist', 'artist type', 'pop' 
song['Tag'] = song['top genre'] + ' ' + song['val'].astype(str) + ' ' + song['nrgy'].astype(str) + ' ' + song['dnce'].astype(str) + ' ' + song['acous'].astype(str) + ' ' + song['spch'].astype(str) + ' ' + song['bpm'].astype(str) + ' ' + song['dB'].astype(str) + ' ' + song['live'].astype(str) + ' ' + song['artist'] + ' ' + song['artist type'] + ' ' + song['pop'].astype(str) 

In [1121]:
song.head()

,title,artist,top genre,bpm,nrgy,dnce,dB,live,val,acous,spch,pop,artist type,Tag
0,STARSTRUKK,3oh!3 katyperry,dancepop,140,81,61,-6,23,23,0,6,70,Duo,dancepop 23 81 61 0 6 140 -6 23 3oh!3 katyperr...
1,My First Kiss,3oh!3 ke$ha,dancepop,138,89,68,-4,36,83,1,8,68,Duo,dancepop 83 89 68 1 8 138 -4 36 3oh!3 ke$ha Du...
2,I Need A Dollar,aloeblacc,popsoul,95,48,84,-7,9,96,20,3,72,Solo,popsoul 96 48 84 20 3 95 -7 9 aloeblacc Solo 72
3,Airplanes,b.o.b hayleywilliamsofparamore,atlhiphop,93,87,66,-4,4,38,11,12,80,Solo,atlhiphop 38 87 66 11 12 93 -4 4 b.o.b hayleyw...
4,Nothin' on You,b.o.b brunomars,atlhiphop,104,85,69,-6,9,74,39,5,79,Solo,atlhiphop 74 85 69 39 5 104 -6 9 b.o.b brunoma...


In [1122]:
#drop the following coloumns 'artist', 'top genre', 'bpm', 'nrgy', 'dnce', 'dB', 'live', 'val', 'dur', 'acous', 'spch', 'pop'
song.drop(['artist', 'top genre', 'bpm', 'nrgy', 'dnce', 'dB', 'live', 'val', 'acous', 'spch', 'pop'], axis=1, inplace=True)

In [1123]:
song.shape


(1037, 3)

In [1124]:

song.drop(['artist type'], axis=1, inplace=True)

song.head()

,title,Tag
0,STARSTRUKK,dancepop 23 81 61 0 6 140 -6 23 3oh!3 katyperr...
1,My First Kiss,dancepop 83 89 68 1 8 138 -4 36 3oh!3 ke$ha Du...
2,I Need A Dollar,popsoul 96 48 84 20 3 95 -7 9 aloeblacc Solo 72
3,Airplanes,atlhiphop 38 87 66 11 12 93 -4 4 b.o.b hayleyw...
4,Nothin' on You,atlhiphop 74 85 69 39 5 104 -6 9 b.o.b brunoma...


In [1125]:
# to remove repeated phrase of same meaning 
ps = PorterStemmer()

def stem(text):
    if isinstance(text, float):
        # Handle the case when text is a float
        return ''  # or any other fallback behavior you prefer

    y = []
    for i in text.split(','):
        y.append(ps.stem(i.strip()))
    return ' '.join(y)

In [1126]:
song['Tag'] = song['Tag'].apply(stem)


In [1127]:
song['Tag'][0]

'dancepop 23 81 61 0 6 140 -6 23 3oh!3 katyperry duo 70'

In [1128]:
#stop word = removal of common words like the are etc
cv = CountVectorizer(max_features=2000,stop_words='english')

In [1129]:
vector = cv.fit_transform(song['Tag']).toarray()

In [1130]:
vector.shape

(1037, 930)

In [1131]:
song.shape

(1037, 2)

In [1132]:
#angle between each vector 
similarity = cosine_similarity(vector)

In [1133]:
similarity

array([[1.        , 0.24019223, 0.        , ..., 0.        , 0.25      ,
        0.        ],
       [0.24019223, 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.19245009, 0.09622504,
        0.10540926],
       ...,
       [0.        , 0.        , 0.19245009, ..., 1.        , 0.08333333,
        0.18257419],
       [0.25      , 0.        , 0.09622504, ..., 0.08333333, 1.        ,
        0.        ],
       [0.        , 0.        , 0.10540926, ..., 0.18257419, 0.        ,
        1.        ]])

In [1134]:
song[song['title'] == 'Somebody Else'].index[0]

648

In [1135]:
def recommend(songs):
    index = song[song['title'] == songs].index[0]
    #arranging the similarity list in descending order
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(song.iloc[i[0]].title)

In [1136]:
recommend('Somebody Else')

Part Of The Band
Ophelia
It's Not Living (If It's Not With You)
Animal
On Top Of The World


In [1137]:
song['title'].values

array(['STARSTRUKK', 'My First Kiss', 'I Need A Dollar', ..., 'Seasonal',
       'Comfort', 'Phase'], dtype=object)

In [1138]:
song.to_dict()

{'title': {0: 'STARSTRUKK',
  1: 'My First Kiss',
  2: 'I Need A Dollar',
  3: 'Airplanes',
  4: "Nothin' on You",
  5: 'Magic',
  6: 'The Time (Dirty Bit)',
  7: 'Imma Be',
  8: 'Talking to the Moon',
  9: 'Just the Way You Are',
  10: 'Teach Me How to Dougie',
  11: 'Forget You',
  12: 'Deuces',
  13: 'Memories',
  14: "Gettin' Over You",
  15: 'All I Do Is Win',
  16: 'Over',
  17: 'Find Your Love',
  18: 'Barbra Streisand (Radio Edit)',
  19: 'Stereo Love - Radio Edit',
  20: 'Pack Up',
  21: 'Starry Eyed',
  22: 'Love The Way You Lie',
  23: 'Not Afraid',
  24: 'I Like It',
  25: 'Kickstarts',
  26: 'Like A G6',
  27: "Club Can't Handle Me",
  28: 'Hot - Malibu Breeze Radio Edit',
  29: 'Replay',
  30: 'Whatcha Say',
  31: "Ridin' Solo",
  32: 'In My Head',
  33: 'Down',
  34: 'Do You Remember',
  35: 'Young Forever',
  36: 'Heartbreak Warfare',
  37: 'Half of My Heart',
  38: 'Baby',
  39: 'California Gurls',
  40: 'Teenage Dream',
  41: 'Commander',
  42: 'TiK ToK',
  43: 'Your 

In [1139]:
song[song['title'] == "Yellow"].index[0]

949

In [1140]:
#show the tag coloumn
song['Tag']


0       dancepop 23 81 61 0 6 140 -6 23 3oh!3 katyperr...
1       dancepop 83 89 68 1 8 138 -4 36 3oh!3 ke$ha du...
2        popsoul 96 48 84 20 3 95 -7 9 aloeblacc  solo 72
3       atlhiphop 38 87 66 11 12 93 -4 4 b.o.b hayleyw...
4       atlhiphop 74 85 69 39 5 104 -6 9 b.o.b brunoma...
                              ...                        
1032    pop 25 36 45 77 4 145 -11 11 adamragsdale  sol...
1033    vancouverindie 41 74 45 0 4 86 -5 12 thebelleg...
1034    irishindie 21 33 33 65 3 84 -9 11 ailbhereddy ...
1035    bubblegrunge 10 23 61 96 3 111 -12 21 petal  b...
1036    alternativeindie 8 43 43 59 4 122 -11 9 atlasi...
Name: Tag, Length: 1037, dtype: object

In [1141]:
#add singer coloumn beside the title coloumn
#song['title'] = song['title'] + ' by ' + singerz


In [1142]:
song.head()

,title,Tag
0,STARSTRUKK,dancepop 23 81 61 0 6 140 -6 23 3oh!3 katyperr...
1,My First Kiss,dancepop 83 89 68 1 8 138 -4 36 3oh!3 ke$ha du...
2,I Need A Dollar,popsoul 96 48 84 20 3 95 -7 9 aloeblacc solo 72
3,Airplanes,atlhiphop 38 87 66 11 12 93 -4 4 b.o.b hayleyw...
4,Nothin' on You,atlhiphop 74 85 69 39 5 104 -6 9 b.o.b brunoma...


In [1143]:
import pickle 
pickle.dump(song.to_dict(), open('song.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))
pickle.dump(singer.to_dict(), open('singer.pkl', 'wb'))



In [1144]:
similarity.shape

(1037, 1037)

In [1145]:
#find index of the song title Phase 
song[song['title'] == 'Phase'].index[0]

1036

In [1146]:
song[song['title'] == 'Phase'].index[0]


1036

In [1147]:
#show the full sng list 
song.head(1030)

,title,Tag
0,STARSTRUKK,dancepop 23 81 61 0 6 140 -6 23 3oh!3 katyperr...
1,My First Kiss,dancepop 83 89 68 1 8 138 -4 36 3oh!3 ke$ha du...
2,I Need A Dollar,popsoul 96 48 84 20 3 95 -7 9 aloeblacc solo 72
3,Airplanes,atlhiphop 38 87 66 11 12 93 -4 4 b.o.b hayleyw...
4,Nothin' on You,atlhiphop 74 85 69 39 5 104 -6 9 b.o.b brunoma...
...,...,...
1025,All Your Days,electropop 10 35 54 44 3 110 -10 11 shallou s...
1026,carry me home,folk 15 33 54 95 3 138 -12 14 sodfven solo 37
1027,Toronto,alternative/indie 21 54 32 43 4 175 -9 14 tusk...
1028,Wild Eyed,metropopolis 8 54 46 84 4 120 -8 10 londongram...


In [1148]:
singer.shape

(1037,)

In [1149]:
song.shape

(1037, 2)

In [1150]:
song.head()


,title,Tag
0,STARSTRUKK,dancepop 23 81 61 0 6 140 -6 23 3oh!3 katyperr...
1,My First Kiss,dancepop 83 89 68 1 8 138 -4 36 3oh!3 ke$ha du...
2,I Need A Dollar,popsoul 96 48 84 20 3 95 -7 9 aloeblacc solo 72
3,Airplanes,atlhiphop 38 87 66 11 12 93 -4 4 b.o.b hayleyw...
4,Nothin' on You,atlhiphop 74 85 69 39 5 104 -6 9 b.o.b brunoma...


In [1151]:
singer.shape

(1037,)

In [1152]:
song.shape

(1037, 2)

In [1153]:
song.head

<bound method NDFrame.head of                 title                                                Tag
0          STARSTRUKK  dancepop 23 81 61 0 6 140 -6 23 3oh!3 katyperr...
1       My First Kiss  dancepop 83 89 68 1 8 138 -4 36 3oh!3 ke$ha du...
2     I Need A Dollar   popsoul 96 48 84 20 3 95 -7 9 aloeblacc  solo 72
3           Airplanes  atlhiphop 38 87 66 11 12 93 -4 4 b.o.b hayleyw...
4      Nothin' on You  atlhiphop 74 85 69 39 5 104 -6 9 b.o.b brunoma...
...               ...                                                ...
1032         Drive-In  pop 25 36 45 77 4 145 -11 11 adamragsdale  sol...
1033         Only One  vancouverindie 41 74 45 0 4 86 -5 12 thebelleg...
1034         Seasonal  irishindie 21 33 33 65 3 84 -9 11 ailbhereddy ...
1035          Comfort  bubblegrunge 10 23 61 96 3 111 -12 21 petal  b...
1036            Phase  alternativeindie 8 43 43 59 4 122 -11 9 atlasi...

[1037 rows x 2 columns]>

In [1154]:
singer.head

<bound method NDFrame.head of 0                      3OH!3 KatyPerry
1                          3OH!3 Ke$ha
2                           AloeBlacc 
3       B.o.B HayleyWilliamsofParamore
4                      B.o.B BrunoMars
                     ...              
1032                     AdamRagsdale 
1033                     TheBelleGame 
1034                      AilbheReddy 
1035                            Petal 
1036                    AtlasinMotion 
Name: artist, Length: 1037, dtype: object>